# EASER Movie Recommendation

## Library

In [1]:
import os
import sys
import random
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pandas as pd
from scipy import sparse

## Config

In [2]:
import yaml

def load_config(config_file):
    with open(config_file, 'r') as stream:
        try:
            config = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return config

cfg = load_config('config.yaml')

In [3]:
if torch.cuda.is_available():
    print('CUDA is available')
    cfg['device'] = True

device = torch.device('cuda' if cfg['device'] else 'cpu')
device

CUDA is available


device(type='cuda')

In [4]:
random.seed(cfg['seed'])
np.random.seed(cfg['seed'])
torch.manual_seed(cfg['seed'])


In [5]:
raw_data = pd.read_csv(os.path.join(cfg['DATA_DIR'], cfg['data']), header=0, usecols=[0, 1])

In [6]:
df_user_movie = raw_data[['user', 'item']]
df_user_movie

,user,item
0,11,4643
1,11,170
2,11,531
3,11,616
4,11,2140
...,...,...
5154466,138493,44022
5154467,138493,4958
5154468,138493,68319
5154469,138493,40819


In [7]:
def get_count(tp, id):
    playcount_groupbyid = tp[[id]].groupby(id, as_index=False)
    count = playcount_groupbyid.size()
    return count

def filter_triplets(tp, min_uc=5, min_sc=0):
    if min_sc > 0:
        itemcount = get_count(tp, 'item')
        tp = tp[tp['item'].isin(itemcount[itemcount['size'] >= min_sc]['item'])]

    if min_uc > 0:
        usercount = get_count(tp, 'user')
        tp = tp[tp['user'].isin(usercount[usercount['size'] >= min_uc]['user'])]

    usercount, itemcount = get_count(tp, 'user'), get_count(tp, 'item')
    return tp, usercount, itemcount

def split_train_test_proportion(data, test_prop=0.2):
    data_grouped_by_user = data.groupby('user')
    tr_list, te_list = list(), list()

    for _, group in data_grouped_by_user:
        n_items_u = len(group)

        if n_items_u >= 5:
            idx = np.zeros(n_items_u, dtype='bool')
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

            tr_list.append(group[np.logical_not(idx)])
            te_list.append(group[idx])

        else:
            tr_list.append(group)

    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)

    return data_tr, data_te

def numerize(tp, user2id, item2id):
    uid = tp['user'].apply(lambda x: user2id[x])
    sid = tp['item'].apply(lambda x: item2id[x])
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

def denumerize(tp, id2user, id2item):
    user = tp['uid'].apply(lambda x: id2user[x])
    item = tp['sid'].apply(lambda x: id2item[x])
    return pd.DataFrame(data={'user': user, 'item': item}, columns=['user', 'item'])

In [8]:
raw_data, user_activity, item_popularity = filter_triplets(raw_data, min_uc=5, min_sc=10)

print("유저별 리뷰수\n",user_activity)
print("아이템별 리뷰수\n",item_popularity)

유저별 리뷰수
          user  size
0          11   376
1          14   180
2          18    77
3          25    91
4          31   154
...       ...   ...
31355  138473    63
31356  138475   124
31357  138486   137
31358  138492    68
31359  138493   314

[31360 rows x 2 columns]
아이템별 리뷰수
         item   size
0          1  12217
1          2   3364
2          3    734
3          4     43
4          5    590
...      ...    ...
6802  118700     54
6803  118900     60
6804  118997     52
6805  119141    122
6806  119145     78

[6807 rows x 2 columns]


In [9]:
# Shuffle User Indices
unique_uid = user_activity['user'].unique()
unique_sid = item_popularity['item'].unique()
print("(BEFORE) unique_uid:",unique_uid)

np.random.seed(cfg['seed'])
idx_perm = np.random.permutation(unique_uid.size)
unique_uid = unique_uid[idx_perm]
print("(AFTER) unique_uid:",unique_uid)

n_users = unique_uid.size #31360
n_items = unique_sid.size #5000


(BEFORE) unique_uid: [    11     14     18 ... 138486 138492 138493]
(AFTER) unique_uid: [ 81259  11986  67552 ...   3671  69383 103755]


In [10]:
raw_data

,user,item
0,11,4643
1,11,170
2,11,531
3,11,616
4,11,2140
...,...,...
5154466,138493,44022
5154467,138493,4958
5154468,138493,68319
5154469,138493,40819


In [11]:
unique_item = raw_data['item'].unique()


In [12]:
item2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
user2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

pro_dir = os.path.join('pro_sg')

if not os.path.exists(pro_dir):
    os.makedirs(pro_dir)

with open(os.path.join(pro_dir, 'unique_sid.txt'), 'w') as f:
    for sid in unique_sid:
        f.write('%s\n' % sid)

with open(os.path.join(pro_dir, 'unique_uid.txt'), 'w') as f:
    for uid in unique_uid:
        f.write('%s\n' % uid)



In [13]:
num_heldout_users = cfg['FOLD_SIZE']

In [14]:
df_for_ease = numerize(raw_data, user2id, item2id)
df_for_ease

,uid,sid
0,13266,2505
1,13266,109
2,13266,319
3,13266,368
4,13266,1183
...,...,...
5154466,4927,4882
5154467,4927,2652
5154468,4927,5768
5154469,4927,4791


In [15]:
df_for_ease['watched'] = [1] * len(df_for_ease)
df_for_ease

,uid,sid,watched
0,13266,2505,1
1,13266,109,1
2,13266,319,1
3,13266,368,1
4,13266,1183,1
...,...,...,...
5154466,4927,4882,1
5154467,4927,2652,1
5154468,4927,5768,1
5154469,4927,4791,1


In [16]:
pivot_table = df_for_ease.pivot_table(index=["uid"], columns=["sid"], values="watched")
X = pivot_table.to_numpy()
X = np.nan_to_num(X)

In [17]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
from copy import deepcopy
from tqdm import tqdm

In [26]:
class EASER():
    def __init__(self, threshold = 3500, lambdaBB = 500, lambdaCC = 10000, rho = 50000, epochs = 40):
        self.threshold = threshold
        self.lambdaBB = lambdaBB
        self.lambdaCC = lambdaCC
        self.rho = rho
        self.epochs = epochs

    def create_list_feature_pairs(self, XtX):
        AA = np.triu(np.abs(XtX))
        AA[ np.diag_indices(AA.shape[0]) ]=0.0
        ii_pairs = np.where((AA > self.threshold) == True)
        return ii_pairs

    def create_matrix_Z(self, ii_pairs, X):
        MM = np.zeros( (len(ii_pairs[0]), X.shape[1]),    dtype=np.float64)
        MM[np.arange(MM.shape[0]) , ii_pairs[0]   ]=1.0
        MM[np.arange(MM.shape[0]) , ii_pairs[1]   ]=1.0
        CCmask = 1.0-MM
        MM = sparse.csc_matrix(MM.T)
        Z=  X * MM
        Z= (Z == 2.0 )
        Z=Z*1.0
        return Z, CCmask

    def train_higher(self, XtX, XtXdiag, ZtZ, ZtZdiag, CCmask, ZtX):
        ii_diag=np.diag_indices(XtX.shape[0])
        XtX[ii_diag] = XtXdiag + self.lambdaBB
        PP = np.linalg.inv(XtX)
        ii_diag_ZZ=np.diag_indices(ZtZ.shape[0])
        ZtZ[ii_diag_ZZ] = ZtZdiag + self.lambdaCC + self.rho
        QQ=np.linalg.inv(ZtZ)
        CC = np.zeros( (ZtZ.shape[0], XtX.shape[0]),dtype=np.float64 )
        DD = np.zeros( (ZtZ.shape[0], XtX.shape[0]),dtype=np.float64 )
        UU = np.zeros( (ZtZ.shape[0], XtX.shape[0]),dtype=np.float64 )

        for iter in tqdm(range(self.epochs)):
            # learn BB
            XtX[ii_diag] = XtXdiag
            BB= PP.dot(XtX-ZtX.T.dot(CC))
            gamma = np.diag(BB) / np.diag(PP)
            BB-= PP * gamma
            # learn CC
            CC= QQ.dot(ZtX-ZtX.dot(BB) + self.rho * (DD-UU))
            # learn DD
            DD=  CC  * CCmask
            DD= np.maximum(0.0, DD) # if you want to enforce non-negative parameters
            # learn UU (is Gamma in paper)
            UU+= CC-DD

        return BB, DD

    def fit(self, X):
        print(' --- init')
        XtX = (X.T @ X)
        XtXdiag = deepcopy(np.diag(XtX))
        ii_pairs = self.create_list_feature_pairs(XtX)
        Z, CCmask = self.create_matrix_Z(ii_pairs, X)

        ZtZ = (Z.transpose() @ Z)
        ZtZdiag = deepcopy(np.diag(ZtZ))

        ZtX = (Z.transpose() @ X)

        print(' --- iteration start.')
        BB, CC = self.train_higher(XtX, XtXdiag, ZtZ, ZtZdiag, CCmask, ZtX)
        print(' --- iteration end.')

        self.pred = torch.from_numpy(X.dot(BB) + Z.dot(CC))

In [27]:
easer = EASER()
easer.fit(X)

 --- init
 --- iteration start.


100%|██████████| 40/40 [12:01<00:00, 18.03s/it]


 --- iteration end.


In [28]:
X_pred = easer.pred.numpy()

In [29]:
X_pred

array([[ 1.58056073e-01,  5.28399747e-02,  2.39387163e-02, ...,
         1.11409375e-02, -2.24802009e-04, -1.35469688e-02],
       [ 5.68532449e-01,  3.03631808e-01,  1.38871537e-01, ...,
         5.52213249e-03, -8.93537334e-03,  8.85305491e-03],
       [ 6.63922872e-01,  2.53986802e-01, -7.04948510e-03, ...,
        -1.01392330e-02, -1.11417953e-02,  4.66640103e-03],
       ...,
       [ 9.23787623e-01,  2.13695113e-01,  1.13757018e-01, ...,
         1.02159590e-02, -2.83937740e-02,  1.54897074e-02],
       [ 2.10567543e-01,  5.65981935e-02,  1.53640773e-02, ...,
        -2.60040300e-03, -5.30697191e-03,  1.85201416e-03],
       [ 3.59878648e-01,  2.52253654e-01, -4.42624093e-02, ...,
        -2.47139061e-03, -5.82726617e-03, -1.56334916e-02]])

In [30]:
X_pred[X.nonzero()] = -np.inf

In [31]:
X_pred

array([[ 1.58056073e-01,  5.28399747e-02,  2.39387163e-02, ...,
         1.11409375e-02, -2.24802009e-04, -1.35469688e-02],
       [ 5.68532449e-01,            -inf,  1.38871537e-01, ...,
         5.52213249e-03, -8.93537334e-03,  8.85305491e-03],
       [           -inf,            -inf, -7.04948510e-03, ...,
        -1.01392330e-02, -1.11417953e-02,  4.66640103e-03],
       ...,
       [           -inf,  2.13695113e-01,  1.13757018e-01, ...,
         1.02159590e-02, -2.83937740e-02,  1.54897074e-02],
       [ 2.10567543e-01,  5.65981935e-02,  1.53640773e-02, ...,
        -2.60040300e-03, -5.30697191e-03,  1.85201416e-03],
       [ 3.59878648e-01,  2.52253654e-01, -4.42624093e-02, ...,
        -2.47139061e-03, -5.82726617e-03, -1.56334916e-02]])

In [32]:
import bottleneck as bn

In [33]:
top_items_by_user = bn.argpartition(-X_pred, 10, axis=1)[:, :10]
print(top_items_by_user)

[[6173 5936 4101 ... 4735 4882 5737]
 [1048  226  356 ... 4101  931  604]
 [ 105  353 3909 ... 1299  894 3916]
 ...
 [4621   41 2156 ... 1494  146 3994]
 [1400 2304  890 ...  178   31 2670]
 [1949 5461 4000 ... 3240  237  355]]


In [34]:
user_result = []
item_result =[]

for id, top_k in enumerate(top_items_by_user):
    user_result.extend([id] * 10)
    item_result.extend(top_k)

df_user_result = pd.DataFrame(user_result, columns=['uid'])
df_item_result = pd.DataFrame(item_result, columns=['sid'])
df_result = pd.concat([df_user_result, df_item_result], axis=1)

In [35]:
df_result

,uid,sid
0,0,6173
1,0,5936
2,0,4101
3,0,5147
4,0,5514
...,...,...
313595,31359,492
313596,31359,3994
313597,31359,3240
313598,31359,237


In [36]:
id2item = dict((i, sid) for (i, sid) in enumerate(unique_sid.squeeze()))
id2user = dict((i, pid) for (i, pid) in enumerate(unique_uid.squeeze()))


In [37]:
df_infer = denumerize(df_result, id2user, id2item)
df_infer.columns = ['user', 'item']
df_infer = df_infer.sort_values('user')
df_infer

,user,item
132668,11,3996
132669,11,2
132667,11,7438
132666,11,40815
132665,11,32587
...,...,...
49275,138493,551
49276,138493,32587
49277,138493,53125
49279,138493,8970


In [ ]:
df_infer.to_csv(os.path.join('submission_EASER.csv'), index=False)
